In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Building Retrieval Models easily with Merlin Models

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output. Such models can be efficiently served by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine and during inference scoring user embeddings over all indexed item embeddings within the engine.In this notebook we will build a Two-Tower architecture for item retrieval.



### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using the Ali-CCP: Alibaba Click and Conversion Prediction dataset. To download the training and test datasets visit Ali-CCP: Alibaba Click and Conversion Prediction at [tianchi.aliyun.com](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1). We have curated the raw dataset via using this [script]() and generated the parquet files that we will use for this example.

### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Perform prediction for a given user query

### Feature Engineering with NVTabular

In [12]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf
import glob
import gc

import nvtabular as nvt
from nvtabular.ops import *
from example_utils import workflow_fit_transform

from merlin.schema.tags import Tags
from merlin.schema import Schema

import merlin.models.tf as mm
import merlin.models.tf.dataset as tf_dataloader

from merlin.io.dataset import Dataset
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata
from merlin.models.tf.blocks.core.aggregation import CosineSimilarity

import tensorflow as tf

2022-03-22 22:10:59.564492: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 22:11:00.695919: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-03-22 22:11:00.696053: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


First, we define our input and output paths.

In [13]:
train_path = '/workspace/data/train/*.parquet'
test_path = '/workspace/data/test/*.parquet'
output_path = '/workspace/retrieval/processed/'

ETL Workflow

We select only positive interaction rows where `click==1` from the dataset with `Filter()` op.

In [14]:
user_id = ["user_id"] >> Categorify() >> AddMetadata(tags=[Tags.USER_ID, Tags.USER])
item_id = ["item_id"] >> Categorify() >> AddMetadata(tags=[Tags.ITEM_ID, Tags.ITEM]) 

item_features = ["item_category", "item_shop", "item_brand"] \
     >> AddMetadata(tags=[Tags.ITEM]) >> nvt.ops.Categorify()

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
    >> AddMetadata(tags=[Tags.USER]) >> nvt.ops.Categorify()

inputs = user_id + item_id + item_features + user_features + ['click'] 

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow_fit_transform(outputs, train_path, test_path, output_path)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


## Building Two-Tower Model

We will use Two-Tower Model for Item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. It is Thus, these systems are often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). You can read more about two stage Recommender Systems [here](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf). In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [5]:
schema = TensorflowMetadata.from_proto_text_file('/workspace/retrieval/processed/train/').to_merlin_schema()

In [6]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

We expect the label names to be empty.

In [7]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Negative sampling
Many datasets for recommender systems contains implicit feedback, with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [8]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),        
    loss="categorical_crossentropy",  
    samplers=[mm.InBatchSampler()],
    embedding_options = mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)]
)

Let's explain the parameters in the TwoTowerModel():
- no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [9]:
batch_size = 4096

train = Dataset(os.path.join(output_path + '/train/*.parquet'), part_size="500MB")
valid = Dataset(os.path.join(output_path + '/test/*.parquet'), part_size="500MB")

In [10]:
model.set_retrieval_candidates_for_evaluation(train)

opt = tf.keras.optimizers.Adagrad(learning_rate=0.003)
model.compile(optimizer=opt, run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=batch_size, epochs=3)

2022-03-22 21:41:38.726291: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


382/384 [============================>.] - ETA: 0s - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3220 - regularization_loss: 0.0000e+00 - total_loss: 8.3220

2022-03-22 21:41:59.919108: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.


...

384/384 [==============================] - 28s 73ms/step - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3183 - regularization_loss: 0.0000e+00 - total_loss: 8.3183 - val_recall_at_10: 0.0024 - val_ndcg_10: 0.0011 - val_loss: 7.4207 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4207
Epoch 3/3
382/384 [============================>.] - ETA: 0s - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3207 - regularization_loss: 0.0000e+00 - total_loss: 8.3207

2022-03-22 21:42:58.220055: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: /tmp/tmpzlyjm7dk/assets


INFO:tensorflow:Assets written to: /tmp/tmpzlyjm7dk/assets


384/384 [==============================] - 28s 73ms/step - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3177 - regularization_loss: 0.0000e+00 - total_loss: 8.3177 - val_recall_at_10: 0.0024 - val_ndcg_10: 0.0011 - val_loss: 7.4201 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4201


In batch sampling is prone to popularity-bias. since different users might have interacted same items. # in the fit method we are only using the negative sampling in each batch and the final score is avg over all the batches. 
for each batch we aer comparing only the items in the given batch. in the `.evaluate()` we are considering entire item catalog for each positive item. generate the scores for all items and check the position of the positive item in the list of scores.

In [11]:
#model.evaluate(valid, return_dict=True, batch_size=1024)

## Exporting Retrieval Models

#### Save user (query) tower

In [12]:
query_tower = model.retrieval_block.query_block()
query_tower.save('query_tower')

INFO:tensorflow:Assets written to: query_tower/assets


INFO:tensorflow:Assets written to: query_tower/assets


#### Extract and save user features

In [13]:
from merlin.models.utils.dataset import unique_rows_by_features
user_features = unique_rows_by_features(Dataset('/workspace/retrieval/processed/train/*.parquet'), Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [14]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,0,0,1,4,1,3,2,1,0,0,0,0
1,1,109,0,0,0,0,0,0,0,69,131,9
2,2,301,1,1,1,1,1,1,2,57,4709,57
3,3,1876,23,7,2,3,1,1,1,5,63,3
4,4,534,1,2,1,2,1,1,0,40,22,108


In [15]:
user_features.shape

(214994, 12)

In [16]:
user_features.to_parquet('user_features.parquet')

#### Extract and save item features

In [17]:
# train path should be the path for the processed parquet files.
item_features = unique_rows_by_features(Dataset('/workspace/retrieval/processed/train/*.parquet'), Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [18]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,0,0,0,0
1,1,441,432,474
2,2,193,1159,125
3,3,3,1463,872
4,4,282,2479,555


In [19]:
item_features.to_parquet('item_features.parquet')

#### Extract and save item embeddings

In [20]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

INFO:tensorflow:Assets written to: /tmp/tmprh8074_m/assets


INFO:tensorflow:Assets written to: /tmp/tmprh8074_m/assets


In [21]:
item_embs_df

,item_id,item_category,item_shop,item_brand,0,1,2,3,4,5,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,0,0.077548,-0.163916,0.062707,0.145624,-0.033684,-0.151762,...,0.130485,-0.127044,-0.219997,0.066048,-0.143901,0.165059,-0.031064,0.001979,0.079951,-0.269703
1,1,441,432,474,0.026989,-0.001228,0.251484,0.004666,-0.178276,-0.056491,...,-0.165452,0.042577,0.076168,0.043248,-0.046577,0.077427,-0.172864,0.090112,0.086928,-0.187773
2,2,193,1159,125,-0.085535,0.202703,0.152450,0.119700,-0.100729,-0.097800,...,-0.010392,0.190749,-0.049466,0.247747,-0.060253,0.231411,-0.096899,0.082744,-0.045622,-0.065368
3,3,3,1463,872,0.021597,-0.042350,0.151263,0.186874,-0.141412,0.014164,...,0.121136,-0.070175,0.089314,0.236508,-0.263751,0.190519,-0.021059,0.205537,-0.036761,-0.241071
4,4,282,2479,555,-0.112511,-0.152873,0.080364,0.155278,-0.035988,0.103379,...,-0.176815,-0.046868,-0.094745,0.207850,0.050684,0.109785,-0.227710,-0.032801,0.077911,0.113340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568842,3078159,1764,365864,121053,0.205734,0.006296,0.152621,0.123247,-0.031887,-0.131998,...,-0.168921,0.109488,-0.103554,0.186951,-0.094818,0.184309,-0.105248,0.063734,-0.004842,-0.095577
568843,3078210,377,97495,116656,0.004852,-0.187302,0.147719,0.206664,-0.008873,0.051659,...,0.027250,0.041539,-0.050605,0.148319,-0.190860,-0.105288,-0.135447,-0.035621,0.134681,0.055756
568844,3078270,3817,577371,80796,-0.087821,0.018948,0.136071,0.136849,-0.115802,0.042283,...,-0.175624,0.014569,-0.013917,0.213448,-0.085530,0.041547,-0.174109,0.098717,0.149980,-0.173722
568845,3078283,551,201089,85172,0.042491,-0.032788,0.073193,0.185784,-0.072671,0.189408,...,-0.070402,-0.040568,0.127943,0.014509,-0.168617,0.217725,0.054450,0.151972,0.049194,-0.092851


In [22]:
# extract only embedding columns
item_embeddings = item_embs_df.iloc[:, 4:]

In [23]:
item_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.077548,-0.163916,0.062707,0.145624,-0.033684,-0.151762,-0.245548,0.056556,0.144074,0.019507,...,0.130485,-0.127044,-0.219997,0.066048,-0.143901,0.165059,-0.031064,0.001979,0.079951,-0.269703
1,0.026989,-0.001228,0.251484,0.004666,-0.178276,-0.056491,0.183419,0.066827,0.249364,-0.063380,...,-0.165452,0.042577,0.076168,0.043248,-0.046577,0.077427,-0.172864,0.090112,0.086928,-0.187773
2,-0.085535,0.202703,0.152450,0.119700,-0.100729,-0.097800,-0.163123,-0.126596,0.140755,-0.154041,...,-0.010392,0.190749,-0.049466,0.247747,-0.060253,0.231411,-0.096899,0.082744,-0.045622,-0.065368
3,0.021597,-0.042350,0.151263,0.186874,-0.141412,0.014164,-0.032892,-0.215597,0.255728,0.021648,...,0.121136,-0.070175,0.089314,0.236508,-0.263751,0.190519,-0.021059,0.205537,-0.036761,-0.241071
4,-0.112511,-0.152873,0.080364,0.155278,-0.035988,0.103379,0.018039,0.096056,0.050665,-0.105511,...,-0.176815,-0.046868,-0.094745,0.207850,0.050684,0.109785,-0.227710,-0.032801,0.077911,0.113340


In [24]:
item_embeddings.to_parquet('item_embeddings')